In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

m = Model ('Investment')

#Read and define variables
df = pd.read_csv('investment.csv')
M, N = df.shape

p = df['Buying Price']
q = df['Current Price']
r = df['Expected Future Price']
s = df['Current Share']

# Setting flag for those stocks whose Current price is greater than Buying Price
# to determine capital gains tax
df['flag'] = 0
df.loc[df['Current Price'] > df['Buying Price'], 'flag'] = 1

print (df)

Restricted license - for non-production use only - expires 2023-10-25
  Stock  Buying Price  Current Share  Current Price  Expected Future Price  \
0    S1           1.2           1000            2.1                    2.0   
1    S2           2.1           1000            3.2                    3.7   
2    S3           3.2           1000            4.1                    5.2   
3    S4           4.1           1000            5.1                    7.1   
4    S5           4.5           1000            6.7                    9.1   

   flag  
0     1  
1     1  
2     1  
3     1  
4     1  


In [2]:
# Creating variable to optimize based on the question
# Please note that Number of Shares to sell has been set as an integer value as in most stock exchanges,
# buying and selling of fractional shares are not equipped
x = m.addVars(M, vtype = GRB.INTEGER, name = "Stock")

# Objective to maximize
m.setObjective(quicksum((s[i] - x[i])*r[i] for i in range(M)),GRB.MAXIMIZE)

# Constraints below
f = df['flag']
m.addConstr( quicksum(0.99*x[i]*q[i] - (x[i]*q[i] - x[i]*p[i])*f[i]*0.30 for i in range(M))  >= 9000, name = 'Transacted Value')
for i in range(M):
    m.addConstr( s[i] - x[i] >= 0 )
    m.addConstr( x[i] >= 0)

# Solving the model
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 11 rows, 5 columns and 15 nonzeros
Model fingerprint: 0xbc4d7746
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 9e+03]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 14199.200000

Root relaxation: objective 1.535679e+04, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15356.7909    0    1 14199.2000 15356.7909  8.15%     -    0s
H    0     0                    15355.100000 15356.790

In [3]:
#  Print optimal solutions and optimal value
print("Number of shares of each stock to sell:")
for v in m.getVars():
    print(v.VarName, v.x)
    
print('Obj:', m.objVal)

Number of shares of each stock to sell:
Stock[0] 999.0
Stock[1] 1000.0
Stock[2] 1000.0
Stock[3] 118.0
Stock[4] 1.0
Obj: 15355.1
